In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%tensorflow_version 1.x
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm
from glob import glob
import time, gc
import cv2
from tensorflow import keras
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import clone_model
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization, Input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib import pyplot as plt
import seaborn as sns
import albumentations as A

Using TensorFlow backend.


In [0]:
TRAIN = ['drive/My Drive/Neural NotWork/Data/train_image_data_0.parquet',
         'drive/My Drive/Neural NotWork/Data/train_image_data_1.parquet',
         'drive/My Drive/Neural NotWork/Data/train_image_data_2.parquet',
         'drive/My Drive/Neural NotWork/Data/train_image_data_3.parquet']

In [0]:
TEST =  ['drive/My Drive/Neural NotWork/Data/test_image_data_0.parquet',
         'drive/My Drive/Neural NotWork/Data/test_image_data_1.parquet',
         'drive/My Drive/Neural NotWork/Data/test_image_data_2.parquet',
         'drive/My Drive/Neural NotWork/Data/test_image_data_3.parquet']

In [0]:
train_df_ = pd.read_csv('drive/My Drive/Neural NotWork/Data/train.csv')
sample_sub_df = pd.read_csv('drive/My Drive/Neural NotWork/Data/sample_submission.csv')

# not really useful, just for human understanding
class_map_df = pd.read_csv('drive/My Drive/Neural NotWork/Data/class_map.csv')

In [0]:
HEIGHT = 137
WIDTH = 236

In [0]:
def resize(df, size=64, need_progress_bar=True):
    resized = {}
    resize_size=56
    angle=0
    if need_progress_bar:
        for i in tqdm(range(df.shape[0])):
            #image = cv2.resize(df.loc[df.index[i]].values.reshape(137,236),(size,size),None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
            image=df.loc[df.index[i]].values.reshape(137,236)
            #Centering
            image_center = tuple(np.array(image.shape[1::-1]) / 2)
            matrix = cv2.getRotationMatrix2D(image_center, angle, 1.0)
            image = cv2.warpAffine(image, matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR,
                            borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
            #Scaling
            matrix = cv2.getRotationMatrix2D(image_center, 0, 1.0)
            image = cv2.warpAffine(image, matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR,
                            borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
            # #Removing Blur
            # aug = A.GaussianBlur(p=1.0)
            # image = aug(image=image)['image']
            # #Noise Removing
            # augNoise=A.MultiplicativeNoise(p=1.0)
            # image = augNoise(image=image)['image']
            # #Removing Distortion
            # augDist=A.ElasticTransform(sigma=50, alpha=1, alpha_affine=10, p=1.0)
            # image = augDist(image=image)['image']
            #Brightness
            augBright=A.RandomBrightnessContrast(p=1.0)
            image = augBright(image=image)['image']
            _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

            idx = 0 
            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            for cnt in contours:
                idx += 1
                x,y,w,h = cv2.boundingRect(cnt)
                ls_xmin.append(x)
                ls_ymin.append(y)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)

            roi = image[ymin:ymax,xmin:xmax]
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            #image=affine_image(image)
            #image= crop_resize(image)
            #image = cv2.resize(image,(size,size),interpolation=cv2.INTER_AREA)
            #image=resize_image(image,(64,64))
            #image = cv2.resize(image,(size,size),interpolation=cv2.INTER_AREA)
            #gaussian_3 = cv2.GaussianBlur(image, (5,5), cv2.BORDER_DEFAULT) #unblur
            #image = cv2.addWeighted(image, 1.5, gaussian_3, -0.5, 0, image)
            #kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
            #image = cv2.filter2D(image, -1, kernel)
            #ret,image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
            resized[df.index[i]] = resized_roi.reshape(-1)
    else:
        for i in range(df.shape[0]):
            #image = cv2.resize(df.loc[df.index[i]].values.reshape(137,236),(size,size),None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
            image=df.loc[df.index[i]].values.reshape(137,236)
            image_center = tuple(np.array(image.shape[1::-1]) / 2)
            matrix = cv2.getRotationMatrix2D(image_center, angle, 1.0)
            image = cv2.warpAffine(image, matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR,
                            borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
            matrix = cv2.getRotationMatrix2D(image_center, 0, 1.0)
            image = cv2.warpAffine(image, matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR,
                            borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
            #Removing Blur
            #aug = A.GaussianBlur(p=1.0)
            #image = aug(image=image)['image']
            #Noise Removing
            #augNoise=A.MultiplicativeNoise(p=1.0)
            #image = augNoise(image=image)['image']
            #Removing Distortion
            #augDist=A.ElasticTransform(sigma=50, alpha=1, alpha_affine=10, p=1.0)
            #image = augDist(image=image)['image']
            #Brightness
            augBright=A.RandomBrightnessContrast(p=1.0)
            image = augBright(image=image)['image']
            _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

            idx = 0 
            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            for cnt in contours:
                idx += 1
                x,y,w,h = cv2.boundingRect(cnt)
                ls_xmin.append(x)
                ls_ymin.append(y)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)

            roi = image[ymin:ymax,xmin:xmax]
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            #image=affine_image(image)
            #image= crop_resize(image)
            #image = cv2.resize(image,(size,size),interpolation=cv2.INTER_AREA)
            #image=resize_image(image,(64,64))
            #image = cv2.resize(image,(size,size),interpolation=cv2.INTER_AREA)
            #gaussian_3 = cv2.GaussianBlur(image, (5,5), cv2.BORDER_DEFAULT) #unblur
            #image = cv2.addWeighted(image, 1.5, gaussian_3, -0.5, 0, image)
            #kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
            #image = cv2.filter2D(image, -1, kernel)
            #ret,image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
            resized[df.index[i]] = resized_roi.reshape(-1)
    resized = pd.DataFrame(resized).T
    return resized

In [0]:
def get_dummies(df):
    cols = []
    for col in df:
        cols.append(pd.get_dummies(df[col].astype(str)))
    return pd.concat(cols, axis=1)

In [0]:
from keras.layers import (
    Input,
    BatchNormalization,
    Activation, Dense, Dropout,
    Convolution2D, MaxPooling2D, ZeroPadding2D
)
from keras.models import Model
from keras.engine import Layer
from keras import backend as K


def theano_multinomial(n, pvals, seed):
    rng = RandomStreams(seed)
    return rng.multinomial(n=n, pvals=pvals, dtype='float32')

def tensorflow_categorical(count, seed):
    assert count > 0
    arr = [1.] + [.0 for _ in range(count-1)]
    return tf.random_shuffle(arr, seed)

# Returns a random array [x0, x1, ...xn] where one is 1 and the others
# are 0. Ex: [0, 0, 1, 0].
def rand_one_in_array(count, seed=None):
    if seed is None:
        seed = np.random.randint(1, 10e6)
    return tensorflow_categorical(count=count, seed=seed)

class JoinLayer(Layer):
    '''
    This layer will behave as Merge(mode='ave') during testing but
    during training it will randomly select between using local or
    global droppath and apply the average of the paths alive after
    aplying the drops.
    - Global: use the random shared tensor to select the paths.
    - Local: sample a random tensor to select the paths.
    '''

    def __init__(self, drop_p, is_global, global_path, force_path, **kwargs):
        #print "init"
        self.p = 1. - drop_p
        self.is_global = is_global
        self.global_path = global_path
        self.uses_learning_phase = True
        self.force_path = force_path
        super(JoinLayer, self).__init__(**kwargs)

    def get_config(self):
        # For serialization with 'custom_objects'
        config = super().get_config()
        config['idx_init'] = self.idx_init
        config['idx_end'] = self.idx_end
        return config

    def build(self, input_shape):
        #print("build")
        self.average_shape = list(input_shape[0])[1:]

    def _random_arr(self, count, p):
        return K.random_binomial((count,), p=p)

    def _arr_with_one(self, count):
        return rand_one_in_array(count=count)

    def _gen_local_drops(self, count, p):
        # Create a local droppath with at least one path
        arr = self._random_arr(count, p)
        drops = K.switch(
            K.any(arr),
            arr,
            self._arr_with_one(count)
        )
        return drops

    def _gen_global_path(self, count):
        return self.global_path[:count]

    def _drop_path(self, inputs):
        count = len(inputs)
        drops = K.switch(
            self.is_global,
            self._gen_global_path(count),
            self._gen_local_drops(count, self.p)
        )
        ave = K.zeros(shape=self.average_shape)
        for i in range(0, count):
            ave += inputs[i] * drops[i]
        sum = K.sum(drops)
        # Check that the sum is not 0 (global droppath can make it
        # 0) to avoid divByZero
        ave = K.switch(
            K.not_equal(sum, 0.),
            ave/sum,
            ave)
        return ave

    def _ave(self, inputs):
        ave = inputs[0]
        for input in inputs[1:]:
            ave += input
        ave /= len(inputs)
        return ave

    def call(self, inputs, mask=None):
        #print("call")
        if self.force_path:
            output = self._drop_path(inputs)
        else:
            output = K.in_train_phase(self._drop_path(inputs), self._ave(inputs))
        return output

    def get_output_shape_for(self, input_shape):
        #print("get_output_shape_for", input_shape)
        return input_shape[0]

class JoinLayerGen:
    '''
    JoinLayerGen will initialize seeds for both global droppath
    switch and global droppout path.
    These seeds will be used to create the random tensors that the
    children layers will use to know if they must use global droppout
    and which path to take in case it is.
    '''

    def __init__(self, width, global_p=0.5, deepest=False):
        self.global_p = global_p
        self.width = width
        self.switch_seed = np.random.randint(1, 10e6)
        self.path_seed = np.random.randint(1, 10e6)
        self.deepest = deepest
        if deepest:
            self.is_global = K.variable(1.)
            self.path_array = K.variable([1.] + [.0 for _ in range(width-1)])
        else:
            self.is_global = self._build_global_switch()
            self.path_array = self._build_global_path_arr()

    def _build_global_path_arr(self):
        # The path the block will take when using global droppath
        return rand_one_in_array(seed=self.path_seed, count=self.width)

    def _build_global_switch(self):
        # A randomly sampled tensor that will signal if the batch
        # should use global or local droppath
        return K.equal(K.random_binomial((), p=self.global_p, seed=self.switch_seed), 1.)

    def get_join_layer(self, drop_p):
        global_switch = self.is_global
        global_path = self.path_array
        return JoinLayer(drop_p=drop_p, is_global=global_switch, global_path=global_path, force_path=self.deepest)

def fractal_conv(filter, nb_row, nb_col, dropout=None):
    def f(prev):
        conv = prev
        conv = Convolution2D(filter, nb_row=nb_col, nb_col=nb_col, init='he_normal', border_mode='same')(conv)
        if dropout:
            conv = Dropout(dropout)(conv)
        conv = BatchNormalization(mode=0, axis= -1)(conv)
        conv = Activation('relu')(conv)
        return conv
    return f

# XXX_ It's not clear when to apply Dropout, the paper cited
# (arXiv:1511.07289) uses it in the last layer of each stack but in
# the code gustav published it is in each convolution block so I'm
# copying it.
def fractal_block(join_gen, c, filter, nb_col, nb_row, drop_p, dropout=None):
    def f(z):
        columns = [[z] for _ in range(c)]
        last_row = 2**(c-1) - 1
        for row in range(2**(c-1)):
            t_row = []
            for col in range(c):
                prop = 2**(col)
                # Add blocks
                if (row+1) % prop == 0:
                    t_col = columns[col]
                    t_col.append(fractal_conv(filter=filter,
                                              nb_col=nb_col,
                                              nb_row=nb_row,
                                              dropout=dropout)(t_col[-1]))
                    t_row.append(col)
            # Merge (if needed)
            if len(t_row) > 1:
                merging = [columns[x][-1] for x in t_row]
                merged  = join_gen.get_join_layer(drop_p=drop_p)(merging)
                for i in t_row:
                    columns[i].append(merged)
        return columns[0][-1]
    return f

def fractal_net(b, c, conv, drop_path, global_p=0.5, dropout=None, deepest=False):
    '''
    Return a function that builds the Fractal part of the network
    respecting keras functional model.
    When deepest is set, we build the entire network but set droppath
    to global and the Join masks to [1., 0... 0.] so only the deepest
    column is always taken.
    We don't add the softmax layer here nor build the model.
    '''
    def f(z):
        output = z
        # Initialize a JoinLayerGen that will be used to derive the
        # JoinLayers that share the same global droppath
        join_gen = JoinLayerGen(width=c, global_p=global_p, deepest=deepest)
        for i in range(b):
            (filter, nb_col, nb_row) = conv[i]
            dropout_i = dropout[i] if dropout else None
            output = fractal_block(join_gen=join_gen,
                                   c=c, filter=filter,
                                   nb_col=nb_col,
                                   nb_row=nb_row,
                                   drop_p=drop_path,
                                   dropout=dropout_i)(output)
            output = MaxPooling2D(pool_size=(2,2), strides=(2,2))(output)
        return output
    return f

In [0]:
IMG_SIZE=56
N_CHANNELS=1

In [0]:
from keras.optimizers import SGD, RMSprop, Adam, Nadam
import tensorflow as tf
def build_network(deepest=False):
    dropout = [0., 0.1, 0.2, 0.3, 0.4]
    conv = [(64, 3, 3), (128, 3, 3), (256, 3, 3), (512, 3, 3), (512, 2, 2)]
    input= Input(shape = (IMG_SIZE, IMG_SIZE, 1))
    output = fractal_net(
        c=3, b=5, conv=conv,
        drop_path=0.15, dropout=dropout,
        deepest=deepest)(input)
    output = Flatten()(output)
    
    head_root = Dense(168, init='he_normal', activation = 'softmax', name = 'roots')(output)
    head_vowel = Dense(11, init='he_normal', activation = 'softmax', name = 'vowels')(output)
    head_consonant = Dense(7, init='he_normal', activation = 'softmax', name = 'consonants')(output)
    model = Model(inputs=input, outputs=[head_root, head_vowel, head_consonant])
    #optimizer = SGD(lr=LEARN_START, momentum=MOMENTUM)
    #optimizer = SGD(lr=LEARN_START, momentum=MOMENTUM, nesterov=True)
    #optimizer = Adam()
    # optimizer = SGD(lr=0.02, momentum=0.9)
    model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [0]:
model = build_network(deepest=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:147: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", kernel_initializer="he_normal")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:150: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(axis=-1)`





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:147: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same", kernel_initializer="he_normal")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:147: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), padding="same", kernel_initializer="he_normal")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:147: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), padding="same", kernel_initializer="he_normal")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:147: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (2, 2), padding="same", kernel_initializer="he_normal")`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(168, activation="softmax", name="roots", kernel_initializer="he_normal")`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(11, activation="softmax", name="vowels", kernel_initializer="he_normal")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(7, activation="softmax", name="consonants", kernel_initializer="he_normal")`
  from ipykernel import kernelapp as app


In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 56, 56, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 56, 56, 64)   640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 56, 56, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 56, 56, 64)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [0]:
learning_rate_reduction_root = ReduceLROnPlateau(monitor='roots_accuracy', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00002)
learning_rate_reduction_vowel = ReduceLROnPlateau(monitor='vowels_accuracy', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00002)
learning_rate_reduction_consonant = ReduceLROnPlateau(monitor='consonants_accuracy', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00002)

In [0]:
#decreased batch size
batch_size = 256
epochs = 30

In [0]:
class MultiOutputDataGenerator(keras.preprocessing.image.ImageDataGenerator):

    def flow(self,
             x,
             y=None,
             batch_size=32,
             shuffle=True,
             sample_weight=None,
             seed=None,
             save_to_dir=None,
             save_prefix='',
             save_format='png',
             subset=None):

        targets = None
        target_lengths = {}
        ordered_outputs = []
        for output, target in y.items():
            if targets is None:
                targets = target
            else:
                targets = np.concatenate((targets, target), axis=1)
            target_lengths[output] = target.shape[1]
            ordered_outputs.append(output)


        for flowx, flowy in super().flow(x, targets, batch_size=batch_size,
                                         shuffle=shuffle):
            target_dict = {}
            i = 0
            for output in ordered_outputs:
                target_length = target_lengths[output]
                target_dict[output] = flowy[:, i: i + target_length]
                i += target_length

            yield flowx, target_dict

In [0]:
## run this before retrain modified model 
keras.backend.backend()

'tensorflow'

In [0]:
histories = []
for i in range(4):
    train_df = pd.merge(pd.read_parquet(f'drive/My Drive/Neural NotWork/Data/train_image_data_{i}.parquet'), train_df_, on='image_id').drop(['image_id'], axis=1)
    # Visualize few samples of current training dataset
    # fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(16, 8))
    # count=0
    # for row in ax:
    #     for col in row:
    #         col.imshow(resize(train_df.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic'], axis=1).iloc[[count]], need_progress_bar=False).values.reshape(IMG_SIZE, IMG_SIZE))
    #         count += 1
    # plt.show()
    
    X_train = train_df.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic', 'grapheme'], axis=1)
    X_train = resize(X_train)/255
    
    # CNN takes images in shape `(batch_size, h, w, channels)`, so reshape the images
    X_train = X_train.values.reshape(-1, IMG_SIZE, IMG_SIZE, N_CHANNELS)
    
    Y_train_root = pd.get_dummies(train_df['grapheme_root']).values
    Y_train_vowel = pd.get_dummies(train_df['vowel_diacritic']).values
    Y_train_consonant = pd.get_dummies(train_df['consonant_diacritic']).values

    print(f'Training images: {X_train.shape}')
    print(f'Training labels root: {Y_train_root.shape}')
    print(f'Training labels vowel: {Y_train_vowel.shape}')
    print(f'Training labels consonants: {Y_train_consonant.shape}')

    # Divide the data into training and validation set
    x_train, x_test, y_train_root, y_test_root, y_train_vowel, y_test_vowel, y_train_consonant, y_test_consonant = train_test_split(X_train, Y_train_root, Y_train_vowel, Y_train_consonant, test_size=0.08, random_state=124)
    del train_df
    del X_train
    del Y_train_root, Y_train_vowel, Y_train_consonant

    # Data augmentation for creating more training data
    datagen = MultiOutputDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=8,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.15, # Randomly zoom image 
        width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


    # This will just calculate parameters required to augment the given data. This won't perform any augmentations
    datagen.fit(x_train)

    # Fit the model
    history = model.fit_generator(datagen.flow(x_train, {'roots': y_train_root, 'vowels': y_train_vowel, 'consonants': y_train_consonant}, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test, [y_test_root, y_test_vowel, y_test_consonant]), 
                              steps_per_epoch=x_train.shape[0] // batch_size, 
                              callbacks=[learning_rate_reduction_root, learning_rate_reduction_vowel, learning_rate_reduction_consonant])

    histories.append(history)
    
    # Delete to reduce memory usage
    del x_train
    del x_test
    del y_train_root
    del y_test_root
    del y_train_vowel
    del y_test_vowel
    del y_train_consonant
    del y_test_consonant
    gc.collect()


Training images: (50210, 56, 56, 1)
Training labels root: (50210, 168)
Training labels vowel: (50210, 11)
Training labels consonants: (50210, 7)


Epoch 1/30
180/180 [==============================] - 123s 685ms/step - loss: 8.5066 - roots_loss: 4.9190 - vowels_loss: 2.2159 - consonants_loss: 1.3716 - roots_acc: 0.0263 - vowels_acc: 0.2059 - consonants_acc: 0.6132 - val_loss: 8.7056 - val_roots_loss: 4.8569 - val_vowels_loss: 2.2456 - val_consonants_loss: 1.6030 - val_roots_acc: 0.0256 - val_vowels_acc: 0.2074 - val_consonants_acc: 0.1424


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1379: RuntimeWarning: Reduce LR on plateau conditioned on metric `roots_accuracy` which is not available. Available metrics are: val_loss,val_roots_loss,val_vowels_loss,val_consonants_loss,val_roots_acc,val_vowels_acc,val_consonants_acc,loss,roots_loss,vowels_loss,consonants_loss,roots_acc,vowels_acc,consonants_acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1379: RuntimeWarning: Reduce LR on plateau conditioned on metric `vowels_accuracy` which is not available. Available metrics are: val_loss,val_roots_loss,val_vowels_loss,val_consonants_loss,val_roots_acc,val_vowels_acc,val_consonants_acc,loss,roots_loss,vowels_loss,consonants_loss,roots_acc,vowels_acc,consonants_acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1379: RuntimeWarning: Reduce LR on plateau conditioned on met

Epoch 2/30
180/180 [==============================] - 110s 609ms/step - loss: 7.5535 - roots_loss: 4.6753 - vowels_loss: 1.7361 - consonants_loss: 1.1421 - roots_acc: 0.0309 - vowels_acc: 0.3757 - consonants_acc: 0.6232 - val_loss: 8.4500 - val_roots_loss: 4.7166 - val_vowels_loss: 2.3256 - val_consonants_loss: 1.4078 - val_roots_acc: 0.0289 - val_vowels_acc: 0.2452 - val_consonants_acc: 0.4663
Epoch 3/30
180/180 [==============================] - 109s 606ms/step - loss: 6.7854 - roots_loss: 4.5079 - vowels_loss: 1.2645 - consonants_loss: 1.0130 - roots_acc: 0.0408 - vowels_acc: 0.5554 - consonants_acc: 0.6493 - val_loss: 7.0685 - val_roots_loss: 4.4300 - val_vowels_loss: 1.5202 - val_consonants_loss: 1.1184 - val_roots_acc: 0.0388 - val_vowels_acc: 0.4272 - val_consonants_acc: 0.6077
Epoch 4/30
180/180 [==============================] - 109s 606ms/step - loss: 5.8244 - roots_loss: 4.0660 - vowels_loss: 0.9459 - consonants_loss: 0.8125 - roots_acc: 0.0725 - vowels_acc: 0.6838 - consona


Training images: (50210, 56, 56, 1)
Training labels root: (50210, 168)
Training labels vowel: (50210, 11)
Training labels consonants: (50210, 7)
Epoch 1/30
180/180 [==============================] - 110s 609ms/step - loss: 0.8387 - roots_loss: 0.4679 - vowels_loss: 0.2131 - consonants_loss: 0.1578 - roots_acc: 0.8696 - vowels_acc: 0.9392 - consonants_acc: 0.9526 - val_loss: 0.4976 - val_roots_loss: 0.3064 - val_vowels_loss: 0.1049 - val_consonants_loss: 0.0863 - val_roots_acc: 0.9124 - val_vowels_acc: 0.9739 - val_consonants_acc: 0.9766
Epoch 2/30
180/180 [==============================] - 109s 606ms/step - loss: 0.7275 - roots_loss: 0.3961 - vowels_loss: 0.1931 - consonants_loss: 0.1383 - roots_acc: 0.8887 - vowels_acc: 0.9449 - consonants_acc: 0.9575 - val_loss: 0.5801 - val_roots_loss: 0.3471 - val_vowels_loss: 0.1307 - val_consonants_loss: 0.1023 - val_roots_acc: 0.9064 - val_vowels_acc: 0.9661 - val_consonants_acc: 0.9736
Epoch 3/30
180/180 [==============================] - 109s


Training images: (50210, 56, 56, 1)
Training labels root: (50210, 168)
Training labels vowel: (50210, 11)
Training labels consonants: (50210, 7)
Epoch 1/30
180/180 [==============================] - 110s 609ms/step - loss: 0.6659 - roots_loss: 0.3713 - vowels_loss: 0.1711 - consonants_loss: 0.1236 - roots_acc: 0.9002 - vowels_acc: 0.9506 - consonants_acc: 0.9647 - val_loss: 0.5166 - val_roots_loss: 0.3323 - val_vowels_loss: 0.1051 - val_consonants_loss: 0.0792 - val_roots_acc: 0.9096 - val_vowels_acc: 0.9754 - val_consonants_acc: 0.9786
Epoch 2/30
180/180 [==============================] - 109s 606ms/step - loss: 0.5764 - roots_loss: 0.3122 - vowels_loss: 0.1546 - consonants_loss: 0.1096 - roots_acc: 0.9122 - vowels_acc: 0.9552 - consonants_acc: 0.9669 - val_loss: 0.4922 - val_roots_loss: 0.2849 - val_vowels_loss: 0.1167 - val_consonants_loss: 0.0907 - val_roots_acc: 0.9263 - val_vowels_acc: 0.9749 - val_consonants_acc: 0.9751
Epoch 3/30
180/180 [==============================] - 109s


Training images: (50210, 56, 56, 1)
Training labels root: (50210, 168)
Training labels vowel: (50210, 11)
Training labels consonants: (50210, 7)
Epoch 1/30
180/180 [==============================] - 110s 610ms/step - loss: 0.6008 - roots_loss: 0.3399 - vowels_loss: 0.1526 - consonants_loss: 0.1084 - roots_acc: 0.9095 - vowels_acc: 0.9564 - consonants_acc: 0.9685 - val_loss: 0.4877 - val_roots_loss: 0.3238 - val_vowels_loss: 0.0850 - val_consonants_loss: 0.0789 - val_roots_acc: 0.9124 - val_vowels_acc: 0.9816 - val_consonants_acc: 0.9813
Epoch 2/30
180/180 [==============================] - 109s 606ms/step - loss: 0.5072 - roots_loss: 0.2746 - vowels_loss: 0.1340 - consonants_loss: 0.0985 - roots_acc: 0.9233 - vowels_acc: 0.9616 - consonants_acc: 0.9708 - val_loss: 0.7108 - val_roots_loss: 0.2659 - val_vowels_loss: 0.3657 - val_consonants_loss: 0.0792 - val_roots_acc: 0.9295 - val_vowels_acc: 0.8476 - val_consonants_acc: 0.9813
Epoch 3/30
180/180 [==============================] - 109s

In [0]:
import os
model_name = 'FractalNet_Model_0302.h5'
save_dir = os.path.join("drive/My Drive/Neural NotWork/", 'saved_models')
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at drive/My Drive/Neural NotWork/saved_models/FractalNet_Model_0302.h5 
